In [1]:
import pandas as pd
import torch.nn as nn
from torchmetrics.functional import pairwise_cosine_similarity
from scipy.special import softmax
import numpy as np

/cluster/raid/home/zhivar.sourati/anaconda3/envs/cpu/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("edu_train.csv")
df.head(3)

,source_article,updated_label,masked_articles
0,"company's slogan ""Expect More. Pay Less.""",appeal to emotion,"company 's slogan "" Expect More . Pay Less . """
1,"The bigger a child's shoe size, the better the...",false causality,"The bigger MSK<0> shoe size , the better MSK<0..."
2,"Since many people believe this, then it must b...",ad populum,"Since many people believe this , then it must ..."


In [3]:
def_df = pd.read_csv("definitions.csv", usecols=list(range(1, 3)))
def_dict = dict(zip(def_df['Original Name'], def_df['Description']))
def_dict

{'faulty generalization': 'an informal fallacy wherein a conclusion is drawn about all or many instances of a phenomenon on the basis of one or a few instances of that phenomenon. is an example of jumping to conclusions.',
 'false causality': 'statement that jumps to a conclusion implying a causal relationship without supporting evidence',
 'circular reasoning': 'when the end of an argument comes back to the beginning without having proven itself.',
 'ad populum': 'a fallacious argument which is based on affirming that something is real or better because the majority thinks so',
 'ad hominem': " instead of addressing someone's argument or position, you irrelevantly attack the person or some aspect of the person who is making the argument.",
 'fallacy of logic': 'an error in the logical structure of an argument.',
 'appeal to emotion': "manipulation of the recipient's emotions in order to win an argument",
 'false dilemma': 'presenting only two options or sides when there are many optio

In [4]:
from sentence_transformers import CrossEncoder
nli_model = CrossEncoder('cross-encoder/nli-distilroberta-base')
def get_nli(sent_a, sent_b):
    scores = nli_model.predict([(sent_a, sent_b)])
    probs = softmax(scores)

    #Convert scores to labels
    label_mapping = ['contradiction', 'entailment', 'neutral']
    labels = [label_mapping[score_max] for score_max in scores.argmax(axis=1)]
    return labels[0], list(zip(probs[0].tolist(), label_mapping))

In [5]:
all_fallacies = df['updated_label'].unique().tolist()
all_fallacies

['appeal to emotion',
 'false causality',
 'ad populum',
 'circular reasoning',
 'fallacy of relevance',
 'faulty generalization',
 'ad hominem',
 'fallacy of extension',
 'equivocation',
 'fallacy of logic',
 'fallacy of credibility',
 'intentional',
 'false dilemma']

### Appeal to Emotions

Maybe can be said that lots of the examples are not actually demonstrating anything agains usual commonsense or daily logic we are used to, it's just they're trying to provoke some sense of `empathy`. 

so maybe we can use a model that is good at identifying this sense. like https://huggingface.co/bdotloh/roberta-base-empathy

In [6]:
sub_df = df[df['updated_label'] == 'appeal to emotion'].sample(10)
print(def_dict['appeal to emotion'])
print('######' * 5)
for sent, masked_sent in zip(sub_df['source_article'].tolist(), sub_df['masked_articles'].tolist()):
    print(sent)
    print(masked_sent)
    print('**' * 10)

manipulation of the recipient's emotions in order to win an argument
##############################
"I am not unmindful that some of you have come here out of great trials and tribulations. Some of you have come fresh from narrow jail cells. And some of you have come from areas where your quest -- quest for freedom left you battered by the storms of persecution and staggered by the winds of police brutality. You have been the veterans of creative suffering. Continue to work with the faith that unearned suffering is redemptive. Go back to Mississippi, go back to Alabama, go back to South Carolina, go back to Georgia, go back to Louisiana, go back to the slums and ghettos of our northern cities, knowing that somehow this situation can and will be changed."  Which appeal is the primary appeal used in this quote?
" I am not unmindful that MSK<0> MSK<1> have MSK<2> here out of great trials and tribulations . MSK<0> MSK<1> have MSK<2> fresh from narrow jail cells . And MSK<0> MSK<1> have MSK

In [7]:
from transformers import RobertaTokenizer, RobertaModel
import torch

checkpoint = "bdotloh/roberta-base-empathy"

tokenizer = RobertaTokenizer.from_pretrained(checkpoint)

model = RobertaModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at bdotloh/roberta-base-empathy were not used when initializing RobertaModel: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at bdotloh/roberta-base-empathy and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def get_embeddings_empathy(texts):
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True)

    outputs = model(**inputs)

    return outputs.pooler_output

In [9]:
text_a = "Just give me a chance to prove it to you. I can’t live this life if you will leave me. My world will stop to turn around. I will not be able to eat nor drink."
text_aa = "You've got to donate to the hockey team. They are helping build schools in Uganda. Imagine all of those poor and hungry kids there."
text_b = "Every time that rooster crows, the sun comes up. The rooster must control the sun."

In [10]:
embeddings_a = get_embeddings_empathy(text_a)
embeddings_aa = get_embeddings_empathy(text_aa)
embeddings_b = get_embeddings_empathy(text_b)

In [15]:
pairwise_cosine_similarity(embeddings_a, embeddings_aa).item(), pairwise_cosine_similarity(embeddings_a, embeddings_b).item()

(0.925356388092041, 0.7271671891212463)

### False Causality

The most important relation that comes to my mind is the relation that different logical parts of the sentences have to each other

This can be modeled with `NLI` task maybe

It can be seen that the NLI models can to some degree distinguish between valid `arguments` and `relevance`

In [15]:
sub_df = df[df['updated_label'] == 'false causality'].sample(10)
print(def_dict['false causality'])
print('######' * 5)
for sent, masked_sent in zip(sub_df['source_article'].tolist(), sub_df['masked_articles'].tolist()):
    print(sent)
    print(masked_sent)
    print('**' * 10)

statement that jumps to a conclusion implying a causal relationship without supporting evidence
##############################
“President Kumail raised taxes, and then the rate of violent crime went up. Kumail is responsible for the rise in crime.”
MSK<2> MSK<0> raised taxes , and then the rate of violent MSK<1> went up . MSK<0> is responsible for the rise in MSK<1> . MSK<2> 
********************
 
The presence of police at protests cause an escalation of violence. It was the case that at the protest last night attended by uniformed police, there was an escalation of violence. Therefore, police should not be at protests.
MSK<0> cause MSK<1> . MSK<0> was the case that at the MSK<2> last night attended by uniformed MSK<3> , there was MSK<1> . Therefore , MSK<3> should not be at MSK<2> . 
********************
Bush was "determined to knock down Saddam Hussein" because of his "nuclear  bomb potential."
MSK<0> was " determined to knock down Saddam Hussein " because of MSK<0> " nuclear bomb p

In [16]:
sample_a = ["Every severe recession follows a Republican Presidency", "therefore Republicans are the cause of recessions."]
get_nli(*sample_a)

('entailment',
 [(0.11655940860509872, 'contradiction'),
  (0.815058171749115, 'entailment'),
  (0.0683823749423027, 'neutral')])

In [17]:
sample_b = ["I started exercising twice a week, and I just got a babysitting job.", "Exercising leads to new jobs!"]
get_nli(*sample_b)

('neutral',
 [(0.19893452525138855, 'contradiction'),
  (0.020097237080335617, 'entailment'),
  (0.7809682488441467, 'neutral')])

In [18]:
sample_c = ["MSK<2> MSK<0> raised taxes , and then the rate of violent MSK<1> went up .", " MSK<0> is responsible for the rise in MSK<1> . MSK<2>"]
get_nli(*sample_c)

('neutral',
 [(0.02253732457756996, 'contradiction'),
  (0.2717932164669037, 'entailment'),
  (0.7056694626808167, 'neutral')])

In [19]:
sample_d = ["I had a real bad headache, then saw my doctor.  Just by talking with him, my headache started to subside, and I was all better the next day.", "It was well worth the $200 visit fee."]
get_nli(*sample_d)

('neutral',
 [(0.010124985128641129, 'contradiction'),
  (0.0012862965231761336, 'entailment'),
  (0.9885886907577515, 'neutral')])

In [20]:
sample_e = ["The economy continues to grow as the number of likes on my Instagram account continue to increase.", "Clearly, the two are linked."]
get_nli(*sample_e)

('neutral',
 [(0.005462161265313625, 'contradiction'),
  (0.06297237426042557, 'entailment'),
  (0.9315654635429382, 'neutral')])

Also, maybe distinction between different parts of the sentence itself would help us. 

So it's not necessarily about the words being connected to each other. it's about the sentences being connected to each other or 

the possibility of entailment between them.

In [21]:
conclusion_words = ["conclusion", "therefore", "so", "hence", "thus", "as a consequence", "for that reason"]

In [22]:
from nltk.tokenize import sent_tokenize
for sent in [' '.join(parts) for parts in [sample_a, sample_b, sample_c, sample_d, sample_e]]:
    print(sent_tokenize(sent))

['Every severe recession follows a Republican Presidency therefore Republicans are the cause of recessions.']
['I started exercising twice a week, and I just got a babysitting job.', 'Exercising leads to new jobs!']
['MSK<2> MSK<0> raised taxes , and then the rate of violent MSK<1> went up .', 'MSK<0> is responsible for the rise in MSK<1> .', 'MSK<2>']
['I had a real bad headache, then saw my doctor.', 'Just by talking with him, my headache started to subside, and I was all better the next day.', 'It was well worth the $200 visit fee.']
['The economy continues to grow as the number of likes on my Instagram account continue to increase.', 'Clearly, the two are linked.']


In [23]:
import spacy
nlp = spacy.load('en_core_web_sm')

for text in [' '.join(parts) for parts in [sample_a, sample_b, sample_c, sample_d, sample_e]]:
    tokens = nlp(text)

    for sent in tokens.sents:
        print(sent)
        print('$$')
    print('---' * 10)

Every severe recession follows a Republican Presidency therefore Republicans are the cause of recessions.
$$
------------------------------
I started exercising twice a week, and I just got a babysitting job.
$$
Exercising leads to new jobs!
$$
------------------------------
MSK<2> MSK<0> raised taxes , and then the rate of violent MSK<1> went up .  
$$
MSK<0> is responsible for the rise in MSK<1> .
$$
MSK<2>
$$
------------------------------
I had a real bad headache, then saw my doctor.  
$$
Just by talking with him, my headache started to subside, and I was all better the next day.
$$
It was well worth the $200 visit fee.
$$
------------------------------
The economy continues to grow as the number of likes on my Instagram account continue to increase.
$$
Clearly, the two are linked.
$$
------------------------------


### Check if NLI model captures first order logic

`NLI models have no notion of first order logic`

In [24]:
get_nli("All Accords are Hondas. All Hondas are Japanese.", "Therefore, all Accords are Japanese.")

('entailment',
 [(0.011736266314983368, 'contradiction'),
  (0.9390512704849243, 'entailment'),
  (0.04921247065067291, 'neutral')])

In [25]:
get_nli("All Accords are Hondas. All Hondas are Japanese.", "Therefore, all Accords are Korean.")

('contradiction',
 [(0.9902350902557373, 'contradiction'),
  (0.0011597290867939591, 'entailment'),
  (0.008605259470641613, 'neutral')])

In [26]:
get_nli("All borogoves are slithy toves.  All slithy toves are mimsy.", "Therefore, all borogoves are mimsy.")

('entailment',
 [(0.00117793760728091, 'contradiction'),
  (0.9729889631271362, 'entailment'),
  (0.025832997635006905, 'neutral')])

In [27]:
get_nli("All x are y. All y are z.", " Therefore, all x are z."), get_nli("All x are y. All y are z.", " Therefore, all y are z."), get_nli("All x are y. All y are z.", " Therefore, all x are y.")

(('entailment',
  [(0.00480384798720479, 'contradiction'),
   (0.9778172373771667, 'entailment'),
   (0.017378931865096092, 'neutral')]),
 ('entailment',
  [(0.002873871708288789, 'contradiction'),
   (0.9842425584793091, 'entailment'),
   (0.012883516028523445, 'neutral')]),
 ('entailment',
  [(0.0014293943531811237, 'contradiction'),
   (0.9853373169898987, 'entailment'),
   (0.01323334127664566, 'neutral')]))

### ad populum

One thing that is so special about this is the fact that in so many of the sentences, you cannot find the connection 

`expressed explicitly`, so technically there's no issue with them. 

In [28]:
fallacy = all_fallacies[2]
sub_df = df[df['updated_label'] == fallacy].sample(10)
print(def_dict[fallacy])
print('######' * 5)
for sent, masked_sent in zip(sub_df['source_article'].tolist(), sub_df['masked_articles'].tolist()):
    print(sent)
    print(masked_sent)
    print('**' * 10)

a fallacious argument which is based on affirming that something is real or better because the majority thinks so
##############################
Verizon advertising the newest Samsung Galaxy phone that they claim everyone else who has bought it loves the new product.
MSK<0> advertising the newest MSK<1> phone that MSK<1> claim everyone else who has bought MSK<0> loves the new product . 
********************
The newest brand jeans that everyone is wearing are called Blue Dogs. Blue Dogs are being worn by all young teens. Be sure to go out and buy your new jeans at your local mall.
MSK<0> are called MSK<1> . MSK<1> are being worn by all young teens . Be sure to go out and buy MSK<0> at coref2 local mall . 
********************
Fortnite is the best game ever. Everyone is playing it!
MSK<0> is the best game ever . Everyone is playing MSK<0> ! 
********************
Over 5 million Filipinos use our products. It’s time to switch to our brand now!
Over 5 million Filipinos use MSK<0> products .

### circular reasoning

Relation between the two parts are `equal`.

In [33]:
fallacy = all_fallacies[3]
print(fallacy)
sub_df = df[df['updated_label'] == fallacy].sample(10)
print(def_dict[fallacy])
print('######' * 5)
for sent, masked_sent in zip(sub_df['source_article'].tolist(), sub_df['masked_articles'].tolist()):
    print(sent)
    print(masked_sent)
    print('**' * 10)

circular reasoning
when the end of an argument comes back to the beginning without having proven itself.
##############################
President Reagan was a great communicator because he had the gift of talking effectively to the people.
MSK<0> was a great communicator because MSK<0> had the gift of talking effectively to the people . 
********************
Speaker 1: You should trust the Bible because it’s the Word of God.
Speaker 2: How do you know it’s the Word of God?
Speaker 1: Because God tells us it is.
Speaker 2: Where does God tell us this?
Speaker 1: Right here, in the Bible.
MSK<1> : MSK<2> should trust MSK<3> because MSK<3> ’s the MSK<4> of MSK<5> . MSK<1> : How do MSK<2> know MSK<3> ’s the MSK<4> of MSK<5> ? MSK<1> : Because MSK<5> MSK<7> MSK<3> is . MSK<1> : Where does MSK<5> MSK<7> this ? MSK<1> : Right here , in the Bible . 
********************
Sleeping until noon is the worst thing to do because it is bad to sleep late.
MSK<0> until noon is the worst thing to do beca

### fallacy of relevance

Relation between two parts is `irrelevant`


In [37]:
fallacy = all_fallacies[4]
print(fallacy)
sub_df = df[df['updated_label'] == fallacy].sample(10)
try:
    print(def_dict[fallacy])
except: pass
print('######' * 5)
for sent, masked_sent in zip(sub_df['source_article'].tolist(), sub_df['masked_articles'].tolist()):
    print(sent)
    print(masked_sent)
    print('**' * 10)

fallacy of relevance
##############################
"Finish your dinner. There are starving children in Africa." is an example of
" Finish your dinner . There are starving children in Africa . " is an example of 
********************
Child: This fish tastes funny. I don’t want to eat this.
Parent: There are children starving in Africa. Eat your dinner.
MSK<0> : This fish tastes funny . I do n’t want to MSK<1> this . Parent : There are MSK<0> starving in Africa . MSK<1> your dinner . 
********************
Compares minor problems with much more serious crimes
Compares minor problems with much more serious crimes 
********************
Your mom thinks you spend too much time playing video games. You bring up that
she spends too much time shopping.
MSK<0> thinks MSK<0> MSK<1> too much MSK<2> playing video games . MSK<0> bring up that coref1 MSK<1> too much MSK<2> shopping . 
********************
Keeping a dog as a pet is every bit as evil as human slavery.
Keeping a MSK<0> as a MSK<0> is ev

### faulty generalization

relation between the first and second statement is `generalization`


In [38]:
fallacy = all_fallacies[5]
print(fallacy)
sub_df = df[df['updated_label'] == fallacy].sample(10)
print(def_dict[fallacy])
print('######' * 5)
for sent, masked_sent in zip(sub_df['source_article'].tolist(), sub_df['masked_articles'].tolist()):
    print(sent)
    print(masked_sent)
    print('**' * 10)

faulty generalization
an informal fallacy wherein a conclusion is drawn about all or many instances of a phenomenon on the basis of one or a few instances of that phenomenon. is an example of jumping to conclusions.
##############################
If you're going to add that much salt to the soup, you might as well pour the whole box in!
If MSK<0> 're going to add that much salt to the soup , MSK<0> might as well pour the whole box in ! 
********************
If we ban cell phones because they are bad for teenagers next the government will ban all iPods and eventually all video games as well.
If we MSK<0> cell phones because they are bad for teenagers next the government will MSK<0> all iPods and eventually all video games as well . 
********************
Brad's mother asked him why he wasn't trying out for the school play. He responded: ''Because if Tom gets the male lead, which he always does, then I'll only be in the chorus, and it's not worth being in the play if I'm only going to be 

### ad hominem

`toxicity` toward one of the parties involved in the first statement.

In [39]:
fallacy = all_fallacies[6]
print(fallacy)
sub_df = df[df['updated_label'] == fallacy].sample(10)
print(def_dict[fallacy])
print('######' * 5)
for sent, masked_sent in zip(sub_df['source_article'].tolist(), sub_df['masked_articles'].tolist()):
    print(sent)
    print(masked_sent)
    print('**' * 10)

ad hominem
 instead of addressing someone's argument or position, you irrelevantly attack the person or some aspect of the person who is making the argument.
##############################
The car was built with cheap parts, so it only goes to show that the car will fall apart.
MSK<0> was built with cheap parts , so MSK<0> only goes to show that MSK<0> will fall apart . 
********************
Layla is a liar! She should not be trusted to be president for student council!
MSK<0> is a liar ! MSK<0> should not be trusted to be president for student council ! 
********************
"I can't trust that doctor's advice on how to raise my child. He has never been a mother.
This is an example of
" MSK<0> ca n't trust MSK<1> advice on how to raise MSK<0> child . MSK<1> has never been a mother . MSK<1> is an example of 
********************
His opinions about sports mean nothing - he can barely tie his own shoes!
MSK<0> opinions about sports mean nothing - MSK<0> can barely tie MSK<0> own shoes ! 

### fallacy of extension

the relation is somehow like the one we saw in faulty generalization. `One of the statements is generalization of other one.`

Again here like the some of the other patterns, there is pattern of `entailment` or `contradiction` that is important. 

In [8]:
fallacy = all_fallacies[7]
print(fallacy)
sub_df = df[df['updated_label'] == fallacy].sample(10)
print(def_dict[fallacy])
print('######' * 5)
for sent, masked_sent in zip(sub_df['source_article'].tolist(), sub_df['masked_articles'].tolist()):
    print(sent)
    print(masked_sent)
    print('**' * 10)

fallacy of extension
attacking an exaggerated or caricatured version of your opponent's position.
##############################
You see someone walking their dog. The dog goes to the bathroom on a lawn but they don’t pick up after their dog. You ask them to clean up after their dog, but they say, “why do you hate dogs?” What fallacy have they committed?
MSK<0> see MSK<1> walking MSK<1> . coref1 goes to the bathroom on a lawn but MSK<1> do n’t pick up after MSK<1> . MSK<0> ask MSK<1> to clean up after MSK<1> , but MSK<1> say , MSK<2> why do MSK<0> hate dogs ? MSK<2> What fallacy have MSK<1> committed ? 
********************
Person A: I think pollution from humans contributes to climate change.
Person B: So, you think humans are directly responsible for extreme weather, like hurricanes, typhoons and droughts.
MSK<0> A : I MSK<1> pollution from MSK<2> contributes to MSK<4> change . MSK<0> B : So , you MSK<1> MSK<2> are directly responsible for extreme MSK<4> , like MSK<3> , MSK<3> and dr

### equivocation

`not clear reasoning at all`

In [7]:
fallacy = all_fallacies[8]
print(fallacy)
sub_df = df[df['updated_label'] == fallacy].sample(10)
print(def_dict[fallacy])
print('######' * 5)
for sent, masked_sent in zip(sub_df['source_article'].tolist(), sub_df['masked_articles'].tolist()):
    print(sent)
    print(masked_sent)
    print('**' * 10)

equivocation
when a key term or phrase in an argument is used in an ambiguous way, with one meaning in one portion of the argument and then another meaning in another portion of the argument.
##############################
The nuthatch was discovered by Tilly Turnow in the woods, while hopping from branch to branch of an elm tree, singing happily.
The nuthatch was discovered by Tilly Turnow in the woods , while hopping from MSK<0> to MSK<0> of an elm tree , singing happily . 
********************
Speaker 1: Did you torture the prisoner?
Speaker 2: No, we just held him under water for a while, and then did a mock hanging.
MSK<2> : Did you torture MSK<1> ? MSK<2> : No , we just held MSK<1> under water for a while , and then did a mock hanging . 
********************
In many cases you can't really tell whether patients are improving because of what the therapist is doing or because of the placebo effect—there's a fuzzy line separating the two--so we must conclude that they are actually th

### fallacy of logic

the relation between two statements is `faulty reasoning`

`we have the premise but the reasoning to get to the conclusion is not correct.`

In [42]:
fallacy = all_fallacies[9]
print(fallacy)
sub_df = df[df['updated_label'] == fallacy].sample(10)
print(def_dict[fallacy])
print('######' * 5)
for sent, masked_sent in zip(sub_df['source_article'].tolist(), sub_df['masked_articles'].tolist()):
    print(sent)
    print(masked_sent)
    print('**' * 10)

fallacy of logic
an error in the logical structure of an argument.
##############################
We live in paradise, but we don't know it.
MSK<0> live in MSK<1> , but MSK<0> do n't know MSK<1> . 
********************
Minds, like rivers, can be broad. The broader the river, the shallower it is. Therefore, the broader the mind, the shallower it is.
MSK<0> , like rivers , can be broad . The MSK<1> MSK<2> , the MSK<3> MSK<2> is . Therefore , the MSK<1> the MSK<0> , the MSK<3> MSK<2> is . 
********************
Hybrid cars are like solar power, full of promise but too expensive. We’ll never be able to build affordable hybrid cars.
MSK<1> are like solar power , full of promise but too expensive . We ’ll never be able to build affordable MSK<1> . 
********************
a mad scientist builds a rocket to the sun but plans to embark at night to avoid being cremated.
a mad scientist builds a rocket to the sun but plans to embark at night to avoid being cremated . 
********************
You let my

### fallacy of credibility

The most prominent sign of using fallacy of credibility is the usage of important people's names

but it doesn't necessarily means that they contain fallacy of credibility if they contain or doesn't contain 

them and also `more important than that` `they have a conclusion without proper support backing it up`.

In [43]:
fallacy = all_fallacies[10]
print(fallacy)
sub_df = df[df['updated_label'] == fallacy].sample(10)
print(def_dict[fallacy])
print('######' * 5)
for sent, masked_sent in zip(sub_df['source_article'].tolist(), sub_df['masked_articles'].tolist()):
    print(sent)
    print(masked_sent)
    print('**' * 10)

fallacy of credibility
attempts to disprove an argument by attacking the character of the speaker.
##############################
Famous actors like Alex Gonzaga and Philip Salvador support Oplan Tokhang, so it must be an effective operation.
Famous actors like Alex Gonzaga and Philip Salvador support Oplan Tokhang , so it must be an effective operation . 
********************
My dad went to school, so he knows everything about the education system.
MSK<0> went to school , so MSK<0> knows everything about the education system . 
********************
"If my years as a Marine taught me anything, it's that caution is the best policy in this sort of situation."?
" If MSK<0> years as a Marine taught MSK<0> anything , it 's that caution is the best policy in this sort of situation . " ? 
********************
“We should abolish the death penalty. Many respected people, such as Imran Kader, have publicly stated their opposition to it.”
MSK<2> We should abolish MSK<0> . MSK<1> , have publicly s

### intentional

The examples are really involved even for human beings. they ask the questions in a way that sometimes needs

commonsense reasoning and sometimes needs stuff even more advanced than commonsense reasoning

In [44]:
fallacy = all_fallacies[11]
print(fallacy)
sub_df = df[df['updated_label'] == fallacy].sample(10)
print(def_dict[fallacy])
print('######' * 5)
for sent, masked_sent in zip(sub_df['source_article'].tolist(), sub_df['masked_articles'].tolist()):
    print(sent)
    print(masked_sent)
    print('**' * 10)

intentional
some intentional (sometimes subconscious) action/choice to incorrectly support an argument
##############################
Barbara believes the marketing agency's office is haunted. The cleaning crew once said they heard a other-worldly noise late at night. Since no one could prove that it wasn't a ghost, the office must be haunted.
Barbara believes MSK<0> is MSK<1> . MSK<2> once said MSK<2> heard MSK<3> late at night . Since no one could prove that MSK<3> was n't a ghost , MSK<0> must be MSK<1> . 
********************
The ideas from that member of the club are wrong, but he isn’t actually a true member of the club so his ideas do not matter.
MSK<0> MSK<1> are wrong , but coref2 is n’t actually a true member of MSK<1> so MSK<0> do not matter . 
********************
ghosts must exist, because nobody has been able to prove that they don't is an example of
MSK<0> must exist , because nobody has been able to prove that MSK<0> do n't is an example of 
********************
Where d

### false dilemma

Depending on the context, if there is the notion of `contradition` or `entailment` between two parts of the sentence, 

there is the evidence of having false dilemma

In [45]:
fallacy = all_fallacies[12]
print(fallacy)
sub_df = df[df['updated_label'] == fallacy].sample(10)
print(def_dict[fallacy])
print('######' * 5)
for sent, masked_sent in zip(sub_df['source_article'].tolist(), sub_df['masked_articles'].tolist()):
    print(sent)
    print(masked_sent)
    print('**' * 10)

false dilemma
presenting only two options or sides when there are many options or sides.
##############################
Either you can save your money to buy a new car for yourself, or you can just take an Uber around for the rest of your life.
Either MSK<0> can save MSK<0> money to buy a new car for MSK<0> , or MSK<0> can just take an Uber around for the rest of MSK<0> life . 
********************
I'm raising funds to help cure XYZ disease. If you don't donate, then you're part of the problem.
I 'm raising funds to help cure XYZ disease . If MSK<0> do n't donate , then MSK<0> 're part of the problem . 
********************
Unless you can close your eyes to abuse, write a check to save this puppy.
Unless MSK<0> can close MSK<0> eyes to abuse , write a check to save this puppy . 
********************
"If you do not submit your requirements today, you should drop your classes with Sir Lean."
" If MSK<0> do not submit MSK<0> requirements today , MSK<0> should drop MSK<0> classes with Sir 

In [83]:
get_nli("We can either stop using cars", "destroy the earth")

('contradiction',
 [(0.9961896538734436, 'contradiction'),
  (0.0004733331152237952, 'entailment'),
  (0.0033370095770806074, 'neutral')])

In [84]:
get_nli("Hillary Clinton wants to take your guns away", "she wants to abolish the Second Amendment")

('neutral',
 [(0.049244653433561325, 'contradiction'),
  (0.009125573560595512, 'entailment'),
  (0.9416298270225525, 'neutral')])

In [85]:
get_nli("Unless you can close your eyes to abuse", " write a check to save this puppy")

('contradiction',
 [(0.8783195614814758, 'contradiction'),
  (0.0006157388561405241, 'entailment'),
  (0.12106466293334961, 'neutral')])

In [86]:
get_nli("I'm raising funds to help cure XYZ disease. If you don't donate", "then you're part of the problem.")

('neutral',
 [(0.07078509032726288, 'contradiction'),
  (0.22080585360527039, 'entailment'),
  (0.7084090709686279, 'neutral')])

In [87]:
get_nli("I thought you cared about other people", "but I didn't see you at the fundraiser for the Children's Hospital")

('neutral',
 [(0.2010866403579712, 'contradiction'),
  (0.001404361566528678, 'entailment'),
  (0.7975090146064758, 'neutral')])

In [89]:
get_nli("you can save your money to buy a new car for yourself", "you can just take an Uber around for the rest of your life.")

('contradiction',
 [(0.9191136956214905, 'contradiction'),
  (0.0011382191441953182, 'entailment'),
  (0.07974815368652344, 'neutral')])